In [1]:
# --- SEGMENT 0: INSTALL DEPENDENCIES ---
print("Installing required libraries for CLIP...")
# We install directly from the official OpenAI GitHub repository and ensure a compatible NumPy
!pip install -q "numpy<2.0"
!pip install -q git+https://github.com/openai/CLIP.git
print("✅ Dependencies installed.")

Installing required libraries for CLIP...
  Preparing metadata (setup.py) ... done
✅ Dependencies installed.


In [2]:
import os
import torch
import clip
from PIL import Image
from google.colab import drive
from tqdm.notebook import tqdm
import shutil

# ==============================================================================
#                                   PART 1: SETUP
# ==============================================================================
print("--- Setting up the environment ---")
drive.mount('/content/drive')

# --- Define Paths ---
CLOTH_DIR = '/content/drive/MyDrive/VirtualFIT_Models/datasets/test/cloth'
ORGANIZED_CLOTH_DIR = '/content/drive/MyDrive/VirtualFIT_Models/datasets/test/organized_cloth'

if not os.path.exists(CLOTH_DIR):
    raise FileNotFoundError(f"The specified folder was not found: {CLOTH_DIR}")

os.makedirs(ORGANIZED_CLOTH_DIR, exist_ok=True)

# --- Load the CLIP Model ---
print("\nLoading the pre-trained CLIP model...")
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)
print("✅ CLIP model loaded successfully.")


# ==============================================================================
#                  PART 2: DEFINE THE CLASSIFICATION FUNCTION
# ==============================================================================

def classify_image(image_path, text_prompts):
    """
    Uses CLIP to find the best text description for an image.
    """
    try:
        image = Image.open(image_path)
        image_input = preprocess(image).unsqueeze(0).to(device)
        text_inputs = clip.tokenize(text_prompts).to(device)

        with torch.no_grad():
            logits_per_image, _ = model(image_input, text_inputs)
            probs = logits_per_image.softmax(dim=-1).cpu().numpy()

        best_match_index = probs.argmax()
        return text_prompts[best_match_index]

    except Exception as e:
        print(f"Could not process {os.path.basename(image_path)}: {e}")
        return None

# ==============================================================================
#                 PART 3: PROCESS ALL IMAGES AND ORGANIZE FILES
# ==============================================================================
print(f"\n--- Analyzing and organizing images from: {CLOTH_DIR} ---")

season_prompts = ["a summer shirt", "a winter sweater", "an autumn jacket", "a spring blouse"]
sleeve_prompts = ["a full-sleeve shirt", "a half-sleeve shirt", "a sleeveless top"]

image_files = [f for f in os.listdir(CLOTH_DIR) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

if not image_files:
    print("No images found in the specified directory.")
else:
    for filename in tqdm(image_files, desc="Processing Images"):
        source_path = os.path.join(CLOTH_DIR, filename)

        season_result = classify_image(source_path, season_prompts)
        season_category = "Unknown_Season"
        if season_result:
            season_category = season_result.split(" ")[1].capitalize()

        sleeve_result = classify_image(source_path, sleeve_prompts)
        sleeve_category = "Unknown_Sleeve"
        if sleeve_result:
            sleeve_category = sleeve_result.split(" ")[1].replace("-", "_").capitalize()

        destination_folder = os.path.join(ORGANIZED_CLOTH_DIR, season_category, sleeve_category)
        os.makedirs(destination_folder, exist_ok=True)
        destination_path = os.path.join(destination_folder, filename)

        shutil.copy(source_path, destination_path)

    print("\n--- ✅ File Organization Complete ---")
    print(f"All images have been analyzed and copied to: {ORGANIZED_CLOTH_DIR}")
    print("\n--- Final Folder Structure ---")
    !tree -d {ORGANIZED_CLOTH_DIR}

--- Setting up the environment ---
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

Loading the pre-trained CLIP model...


100%|████████████████████████████████████████| 338M/338M [00:02<00:00, 160MiB/s]


✅ CLIP model loaded successfully.

--- Analyzing and organizing images from: /content/drive/MyDrive/VirtualFIT_Models/datasets/test/cloth ---


Processing Images:   0%|          | 0/2032 [00:00<?, ?it/s]


--- ✅ File Organization Complete ---
All images have been analyzed and copied to: /content/drive/MyDrive/VirtualFIT_Models/datasets/test/organized_cloth

--- Final Folder Structure ---
/bin/bash: line 1: tree: command not found
